## Przykłady

### Przykład 9 - zewnętrzny, dynamiczny kontekst

In [56]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

loader = TextLoader('DAY7-pliki/adam.md')
doc = loader.load()
chat = ChatOpenAI()

#print(doc[0])
response = chat.invoke([
    SystemMessage(f"Answer questions as truthfully using the context below and nothing more. If you don't know the answer, say \"don't know\".\
        context\###${doc[0].page_content}###"),
    HumanMessage("Kim jest overment?")
])

print(response.content)

Overment to pseudonim Adama.


### Przykład 10 - wybór źródła wiedzy na podstawie prompta

In [74]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage

chat = ChatOpenAI()
question = "Czy MC ma psa?"
sources = [
    {"name": "Adam (overment)", "source": "DAY7-pliki/adam.md"},
    {"name": "Jakub (unknown)", "source": "DAY7-pliki/jakub.md"},
    {"name": "Mateusz (MC)", "source": "DAY7-pliki/mateusz.md"},
]
sources_list = [f"{s['name']} file:{s['source']}" for s in sources]
sources_str = '\n'.join(sources_list)

source = chat.invoke([
    SystemMessage(f"""Pick one of the following sources related to the query and return filename and nothing else.
        Sources###
        {sources_str}
        ###
        Query: {question}\n\n
        Source file name:""")
])

print(source.content)

DAY7-pliki/mateusz.md


### Przykład 11 - tworzenie dokumentów na podstawie fragmentów plików (ang.chunk)

In [90]:
import os
import json
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.docstore.document import Document

loader = TextLoader('DAY7-pliki/docs.md')
doc = loader.load()

# podział pliku na dokumenty według podwójnego znkau nwoej linii
documents = [Document(page_content=content) for content in doc[0].page_content.split("\n\n")]

# przetworzenie dokumentów promptem opisującym ich źródło (w tym przypadku jest to wybór imienia osoby, z którą powiązany jest dany dokument)
model = ChatOpenAI()

system_message = SystemMessage('''Describe the following document with one of the following keywords:
            Mateusz, Jakub, Adam. Return the keyword and nothing else.''')

descriptions = []
for document in documents:
    description = chat.invoke([
        system_message,
        HumanMessage(f"{document.page_content}")
    ])
    descriptions.append(description.content)

for index, description in enumerate(descriptions):
    # Zaktualizuj 'source' w 'metadata' odpowiedniego dokumentu za pomocą opisu
    documents[index].metadata['source'] = description

# Przekształć listę obiektów 'documents' na listę słowników
documents_dict = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]

try:
    with open("DAY7-pliki/docs.json", "w", encoding='utf8') as file:
        json.dump(documents_dict, file, indent=1)
except FileNotFoundError:
    print("Nie znaleziono katalogu 'DAY7-pliki'. Upewnij się, że katalog istnieje.")

TypeError: __init__() got an unexpected keyword argument 'encoding'